In [ ]:
#!pip install selenium

Need to download Chromedriver first. Please vsit this page: https://chromedriver.chromium.org/downloads

In [1]:
import re
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
tech_company_name = pd.read_excel('CompanyName.xlsx')
tech_company_name_df = tech_company_name['Company Name'].str.strip() # remove any white spaces
names = {
    'Company Name': tech_company_name_df
}

tech_name_df = pd.DataFrame(names)
tech_name_df.drop_duplicates(keep = 'first', inplace = True) # in real world situation, you might encounter duplicates
tech_name_df['Company Name'] = tech_name_df['Company Name'].str.lower()
tech_names = tech_name_df['Company Name']
print(tech_names)

0          eisneramper
1         smart design
2    data products llc
3               google
4               amazon
Name: Company Name, dtype: object


In [3]:
driver = webdriver.Chrome() # open up chrome

In [4]:
# go to linkedin sign in page
driver.get("https://www.linkedin.com/checkpoint/rm/sign-in-another-account?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

In [5]:
# locate the username input box
username = driver.find_element(By.ID, "username")

In [6]:
# read my linkedin account username and password
linkedin_account = pd.read_excel('LinkedInAccount.xlsx')

In [7]:
# send my username to the username input box
username.send_keys(linkedin_account['User Name'])

In [8]:
# locate password input box
pword = driver.find_element(By.ID, "password")

In [9]:
# send my password to the password input box
pword.send_keys(linkedin_account['Password'])

In [10]:
# find the element with class name and clikc it
driver.find_element(By.CLASS_NAME, "login__form_action_container ").click()

In [11]:
# function to get the url based on company name
def get_url(search_term):
    template = "https://www.linkedin.com/company/{}/about/"
    search_term = search_term.replace(',', '')
    search_term = search_term.replace('(', '')
    search_term = search_term.replace(')', '-')
    search_term = search_term.replace(' ', '-')
    return (template.format(search_term), search_term)

In [12]:
# define lists for needed data
names = []
descriptions = []
websites = []
industries = []
sizes = []

In [13]:
# see an example url
company_url = get_url("eisneramper")
print(company_url)

('https://www.linkedin.com/company/eisneramper/about/', 'eisneramper')


In [14]:
# for each company, scrape the data we need
for i in tech_names:
    try:    
        company_url = get_url(i)[0]
        driver.get(company_url)
        time.sleep(3)
        src = driver.page_source # store page source
        soup = BeautifulSoup(src, 'lxml') # parse HTML
        name = soup.find('h1')
        description = soup.find('p', class_ = "break-words white-space-pre-wrap t-black--light text-body-medium")
        website = soup.find('a', class_ = 'link-without-visited-state ember-view')
        industry = soup.find('dt', text = re.compile("Industry")).find_next('dd').text.strip()
        size = soup.find('dt', text = re.compile("Company size")).find_next('dd').text.strip()
        
        names.append(name.get_text().strip())
        descriptions.append(description.get_text().strip('\n'))
        websites.append(website.get('href'))
        industries.append(industry)
        sizes.append(size)
        
    except:
        print(i)
        print("name:", len(names))
        print("description:", len(descriptions))
        print("website:", len(websites))
        print("industry:", len(industries))
        print("size:", len(sizes))
    

In [15]:
# make sure we get all the data we need
print(len(names))
print(len(descriptions))
print(len(websites))
print(len(industries))
print(len(sizes))

5
5
5
5
5


In [16]:
# make it into a data frame
company_info = {
    'CompanyName': names,
    'Description': descriptions,
    'Website': websites,
    'Industry': industries,
    'Size': sizes
}

company_info_df = pd.DataFrame(company_info)

In [17]:
company_info_df

,CompanyName,Description,Website,Industry,Size
0,EisnerAmper,"EisnerAmper, one of the largest business consu...",http://www.eisneramper.com,Accounting,"1,001-5,000 employees"
1,Smart Design,Smart Design is a strategic design company tha...,http://smartdesignworldwide.com,Design Services,51-200 employees
2,Data Products LLC,"WHAT WE DO: \nAt our core, we are dedicated to...",http://www.dataproducts.io,Professional Services,11-50 employees
3,Google,A problem isn't truly solved until it's solved...,https://goo.gle/3DLEokh,Software Development,"10,001+ employees"
4,Amazon,Amazon is guided by four principles: customer ...,https://www.aboutamazon.com/,Software Development,"10,001+ employees"


In [ ]:
company_info_df.to_excel('Company Info.xlsx', index = False)